<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [ ]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2024-09-09 21:44:11--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2024-09-09 21:44:11--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2024-09-09 21:44:12--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [ ]:
# Posibles bandas
os.listdir("./songs_dataset/")

['Lil_Wayne.txt',
 'nicki-minaj.txt',
 'michael-jackson.txt',
 'drake.txt',
 'dolly-parton.txt',
 'notorious-big.txt',
 'notorious_big.txt',
 'joni-mitchell.txt',
 'nirvana.txt',
 'eminem.txt',
 'leonard-cohen.txt',
 'dickinson.txt',
 'jimi-hendrix.txt',
 'nursery_rhymes.txt',
 'blink-182.txt',
 'bjork.txt',
 'bruce-springsteen.txt',
 'missy-elliott.txt',
 'bruno-mars.txt',
 'radiohead.txt',
 'cake.txt',
 'kanye-west.txt',
 'nickelback.txt',
 'dr-seuss.txt',
 'adele.txt',
 'lin-manuel-miranda.txt',
 'lady-gaga.txt',
 'beatles.txt',
 'alicia-keys.txt',
 'disney.txt',
 'al-green.txt',
 'bob-marley.txt',
 'kanye.txt',
 'patti-smith.txt',
 'paul-simon.txt',
 'dj-khaled.txt',
 'britney-spears.txt',
 'lorde.txt',
 'ludacris.txt',
 'r-kelly.txt',
 'johnny-cash.txt',
 'bob-dylan.txt',
 'janisjoplin.txt',
 'prince.txt',
 'rihanna.txt',
 'bieber.txt',
 'lil-wayne.txt',
 'Kanye_West.txt',
 'amy-winehouse.txt']

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

<ipython-input-4-91c4bad52ba1>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)


,0
0,"Yesterday, all my troubles seemed so far away"
1,Now it looks as though they're here to stay
2,"Oh, I believe in yesterday Suddenly, I'm not h..."
3,There's a shadow hanging over me.
4,"Oh, yesterday came suddenly Why she had to go ..."


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1846


### 1 - Preprocesamiento

In [ ]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:2]

[['yesterday', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away'],
 ['now', 'it', 'looks', 'as', 'though', "they're", 'here', 'to', 'stay']]

### 2 - Crear los vectores (word2vec)

In [2]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1846


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 445


### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 113208.15625
Loss after epoch 1: 65953.953125
Loss after epoch 2: 65934.265625
Loss after epoch 3: 65711.96875
Loss after epoch 4: 63872.5
Loss after epoch 5: 64142.46875
Loss after epoch 6: 64058.8125
Loss after epoch 7: 64755.4375
Loss after epoch 8: 62585.5625
Loss after epoch 9: 60415.25
Loss after epoch 10: 59823.625
Loss after epoch 11: 58930.1875
Loss after epoch 12: 57726.4375
Loss after epoch 13: 56493.125
Loss after epoch 14: 55842.5625
Loss after epoch 15: 55862.5
Loss after epoch 16: 51695.8125
Loss after epoch 17: 49815.5
Loss after epoch 18: 49568.125
Loss after epoch 19: 48959.75


(156986, 287740)

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('pretty', 0.9087147116661072),
 ('sleep', 0.8649678230285645),
 ('help', 0.8594305515289307),
 ('try', 0.8456019759178162),
 ('cry', 0.8387227654457092),
 ('little', 0.8273633718490601),
 ('not', 0.8181102871894836),
 ('seems', 0.8175203204154968),
 ('twist', 0.8154979348182678),
 ('peace', 0.812263011932373)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('shake', -0.21786975860595703),
 ('our', -0.24495810270309448),
 ('come', -0.2613270878791809),
 ('on', -0.26722386479377747),
 ('bang', -0.27329546213150024),
 ('five', -0.2741836607456207),
 ('six', -0.2774038314819336),
 ('baby', -0.2776353061199188),
 ('work', -0.2790372967720032),
 ('four', -0.2792743146419525)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('five', 0.9786694049835205),
 ('three', 0.9761051535606384),
 ('six', 0.966891348361969),
 ('two', 0.9565720558166504),
 ('seven', 0.954232931137085),
 ('sixty', 0.9003373980522156),
 ('one', 0.816501259803772),
 ('us', 0.7762899398803711),
 ('crying', 0.772737979888916),
 ('strawberry', 0.7672452330589294)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[("can't", 0.9409592747688293),
 ('buy', 0.9399508237838745),
 ('much', 0.8996850848197937),
 ('hide', 0.8465201258659363),
 ('just', 0.8415144085884094)]

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "Key 'diedaa' not present in vocabulary"

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.17736822  0.21460655 -0.05408018  0.06707036  0.13059464 -0.25713933
  0.00711611  0.26743424 -0.03100746  0.259963    0.21179457 -0.08465387
 -0.26272097  0.308386   -0.07934437 -0.13357355  0.2994755   0.18062147
 -0.13111673 -0.11857472  0.03468326  0.1125607   0.08478875  0.08090881
 -0.17622267  0.03122034 -0.08725598  0.23746666 -0.11507358 -0.29119617
 -0.03650665 -0.3322312   0.23262788  0.10366543 -0.18365337  0.08299578
  0.14315338 -0.19761457 -0.00757894  0.1119568  -0.10421463 -0.13083619
 -0.03264419 -0.04687435  0.11330808  0.10231452 -0.16202034 -0.10347028
  0.03659901 -0.08524423 -0.34872082 -0.02154255  0.25938764  0.16540806
  0.0310861   0.09505633  0.24452917 -0.09859891  0.18298703  0.0645447
  0.016967   -0.3259168  -0.04774074 -0.05418535 -0.04338397 -0.09137302
  0.00889874  0.11636022 -0.25662988  0.05426472  0.08915784  0.04541444
  0.20835714 -0.19101828  0.32210803  0.19337991  0.09080223 -0.00362429
 -0.18664253 -0.07053486 -0.1213743  -0.03477469 -0.

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('babe', 0.9132986068725586),
 ('someone', 0.8914992809295654),
 ('nothing', 0.8811879754066467),
 ('need', 0.8758756518363953),
 ("didn't", 0.8689683079719543),
 ("there's", 0.8557919859886169),
 ('you', 0.8447725772857666),
 ('feed', 0.8422104716300964),
 ('somebody', 0.8385539054870605)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('babe', 0.9132986068725586),
 ('someone', 0.8914992809295654),
 ('nothing', 0.8811879754066467),
 ('need', 0.8758756518363953),
 ("didn't", 0.8689682483673096),
 ("there's", 0.8557919859886169),
 ('you', 0.8447725772857666),
 ('feed', 0.8422104716300964),
 ('somebody', 0.8385538458824158),
 ('hope', 0.8361446261405945)]

### 5 - Visualizar agrupación de vectores

In [32]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

Uso como datashet la transcripcion de una clase de Aprendisaje de Maquinas 1

In [2]:
df = pd.read_csv('Clase.txt', sep='/n', header=None)
df.head(10)

C:\Users\germa\AppData\Local\Temp\ipykernel_46076\1727040919.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('Clase.txt', sep='/n', header=None)


,0
0,00:00 Cursos CAPSE: técnico del otro lado de o...
1,02:10 Agustin Lambertini (chat): Buen dia!
2,"03:01 Cursos CAPSE: Bueno, no están solucionan..."
3,03:48 Cursos CAPSE: Bien. esto bueno Bienvenid...
4,04:58 Cursos CAPSE: Tienen vamos a tener ocho ...
5,05:52 Cursos CAPSE: De la materia que se las p...
6,06:37 Cursos CAPSE: Donde se va a estar subien...
7,06:54 Cursos CAPSE: Sacamos todo lo que es rad...
8,08:41 Cursos CAPSE: El cual es una implementac...
9,10:08 Cursos CAPSE: Esto es un trabajo más. Ac...


In [5]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []

for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [23]:
w2v_model2 = Word2Vec(min_count=5,
                     window=3,
                     vector_size=100,
                     negative=20,
                     workers=1,
                     sg=1)

w2v_model2.build_vocab(sentence_tokens)

In [24]:
w2v_model2.train(sentence_tokens,
                 total_examples=w2v_model2.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 203117.046875
Loss after epoch 1: 142662.984375
Loss after epoch 2: 142965.25
Loss after epoch 3: 145236.46875
Loss after epoch 4: 145785.8125
Loss after epoch 5: 146083.125
Loss after epoch 6: 142528.9375
Loss after epoch 7: 127227.375
Loss after epoch 8: 126122.0
Loss after epoch 9: 124298.875
Loss after epoch 10: 124531.0
Loss after epoch 11: 124291.0
Loss after epoch 12: 124541.875
Loss after epoch 13: 122151.125
Loss after epoch 14: 121816.75
Loss after epoch 15: 109293.625
Loss after epoch 16: 103692.75
Loss after epoch 17: 103916.75
Loss after epoch 18: 103726.75
Loss after epoch 19: 102699.0


(186724, 371480)

In [25]:
print("Cantidad de words distintas en el corpus:", len(w2v_model2.wv.index_to_key))

Cantidad de words distintas en el corpus: 526


In [26]:
w2v_model2.wv.most_similar(positive=["modelo"], topn=10)

[('listo', 0.8678217530250549),
 ('había', 0.8675009608268738),
 ('algoritmo', 0.857895016670227),
 ('mejor', 0.8513900637626648),
 ('primer', 0.8483192920684814),
 ('famoso', 0.8464131951332092),
 ('base', 0.8416242599487305),
 ('mi', 0.8402443528175354),
 ('avanzado', 0.8395497798919678),
 ('uso', 0.8394960165023804)]

In [27]:
w2v_model2.wv.most_similar(positive=["inteligencia"], topn=10)

[('vimos', 0.9591383337974548),
 ('visto', 0.9483568668365479),
 ('artificial', 0.9226527214050293),
 ('trabajando', 0.9072145819664001),
 ('otras', 0.8951878547668457),
 ('reacción', 0.8746283054351807),
 ('hemos', 0.8639785647392273),
 ('materias', 0.8623473048210144),
 ('tienes', 0.8621289134025574),
 ('vieron', 0.8570652604103088)]

In [20]:
w2v_model2.wv.most_similar(positive=["artificial"], topn=10)

[('reacción', 0.8810503482818604),
 ('vimos', 0.8804090023040771),
 ('visto', 0.8786309957504272),
 ('trabajando', 0.873639702796936),
 ('vieron', 0.8689830303192139),
 ('inteligencia', 0.8674653768539429),
 ('tienes', 0.8551253080368042),
 ('otras', 0.8366392850875854),
 ('igual', 0.8314890265464783),
 ('hemos', 0.826524555683136)]

In [43]:
w2v_model2.wv.most_similar(positive=["aprendizaje"], topn=10)

[('supervisado', 0.7818177938461304),
 ('profundo', 0.7770248651504517),
 ('máquinas', 0.7729719281196594),
 ('visto', 0.7725295424461365),
 ('automáticos', 0.7710980772972107),
 ('general', 0.7660079598426819),
 ('aprendizajes', 0.7647590041160583),
 ('trabajando', 0.7629148960113525),
 ('materias', 0.7576556205749512),
 ('usar', 0.7556304931640625)]

In [44]:
w2v_model2.wv.most_similar(positive=["40"], topn=10)

[('12', 0.9980653524398804),
 ('26', 0.9974932670593262),
 ('43', 0.996850848197937),
 ('46', 0.9968019127845764),
 ('04', 0.996772825717926),
 ('38', 0.9964684247970581),
 ('21', 0.9962515830993652),
 ('53', 0.9961917996406555),
 ('34', 0.9960583448410034),
 ('54', 0.9960098266601562)]

In [29]:
w2v_model2.wv.most_similar(positive=["clase"], topn=10)

[('materias', 0.9702839255332947),
 ('otras', 0.9663208723068237),
 ('trabajando', 0.9657101631164551),
 ('igual', 0.9447810053825378),
 ('máquina', 0.9322678446769714),
 ('reacción', 0.918692946434021),
 ('principalmente', 0.9163973331451416),
 ('viene', 0.9161229133605957),
 ('ustedes', 0.9143019914627075),
 ('automáticos', 0.9099277853965759)]

Al analizar algunos terminos considerando las 10 palabras mas relacionadas con estos, se observa que algunas de las palabras tienen relacion con el termino analizado, por ejemplo en el caso de la palabra "clase" tiene una alta similaridad con "materias", "maquina" y "automatico", lo cual tiene sentido ya que el texto es una transcripcion de una clase de Aprendizaje de maquina. Tambien se observan palabras con alta similaridad cuya relacion con la palabra clase no es tan evidente, como por ejemplo "igual" o "reaccion".

In [30]:
import plotly.graph_objects as go
import plotly.express as px

In [37]:
vecs, labels = reduce_dimensions(w2v_model2)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [33]:
vecs, labels = reduce_dimensions(w2v_model2,3)



In [34]:

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab")

En los graficos se observa que hay un grupo de terminos alejados del resto, de los cuales gran parte son numeros, esto se debe a que en la transcripcion todas las lineas comienzan con la hora y la frase "Cursos CAPSE", lo que hace que estos terminos tengan alta similaridad entre si.